In [1]:
from pynq import Overlay
ol = Overlay('matrix.bit')

In [2]:
#ol?

In [3]:
dma = ol.axi_dma_0

In [4]:
data_send = dma.sendchannel
data_recv = dma.recvchannel

In [5]:
import numpy as np
import random 

In [6]:
random.seed(2)
data_A = np.random.uniform(low=0, high=10, size=(8, 8)).astype(float)
data_B = np.random.uniform(low=0, high=10, size=(8, 8)).astype(float)

In [7]:
data_A, data_B

(array([[2.55442758, 2.52412382, 3.88096982, 7.14789616, 6.91700525,
         7.35113545, 9.12497269, 7.72726403],
        [6.00185917, 8.18409427, 8.06308615, 6.28970654, 3.07797372,
         6.78414977, 4.15708273, 1.37440004],
        [8.40702691, 5.91890812, 4.93811167, 5.38884031, 9.8871736 ,
         4.69131453, 8.60814821, 7.34646565],
        [1.19577512, 0.28323673, 6.72382984, 8.87116014, 6.0477839 ,
         9.3009059 , 0.13590772, 6.79624819],
        [8.53469262, 1.81478871, 6.83141149, 5.3951494 , 7.81973643,
         6.4202612 , 7.75965555, 8.13325488],
        [2.60541699, 3.98283022, 3.06256559, 1.86904189, 6.58160435,
         2.71300691, 4.36125841, 2.04978484],
        [3.99364208, 8.64185631, 9.68136177, 8.77843342, 8.07349922,
         9.80650943, 1.18234631, 4.78462409],
        [2.10586968, 2.13110648, 4.45772306, 2.7694983 , 7.61192806,
         0.8625916 , 1.48446479, 9.97881665]]),
 array([[2.55960806, 8.20268461, 0.47684678, 2.95886526, 5.66401659,
         

In [9]:
def multiply(matrix_a, matrix_b):
    result_matrix = np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            for k in range(8):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]
                
    return result_matrix

In [32]:
import time
start_time = time.time()
multiply(data_A, data_B) #software
end_time = time.time()
print(end_time - start_time)

0.01158285140991211


In [19]:
from pynq import allocate

In [20]:
data_a_f = data_A.flatten()
data_b_f = data_B.flatten()

data_input = np.concatenate([data_a_f, data_b_f])

In [21]:
data_input

array([2.55442758, 2.52412382, 3.88096982, 7.14789616, 6.91700525,
       7.35113545, 9.12497269, 7.72726403, 6.00185917, 8.18409427,
       8.06308615, 6.28970654, 3.07797372, 6.78414977, 4.15708273,
       1.37440004, 8.40702691, 5.91890812, 4.93811167, 5.38884031,
       9.8871736 , 4.69131453, 8.60814821, 7.34646565, 1.19577512,
       0.28323673, 6.72382984, 8.87116014, 6.0477839 , 9.3009059 ,
       0.13590772, 6.79624819, 8.53469262, 1.81478871, 6.83141149,
       5.3951494 , 7.81973643, 6.4202612 , 7.75965555, 8.13325488,
       2.60541699, 3.98283022, 3.06256559, 1.86904189, 6.58160435,
       2.71300691, 4.36125841, 2.04978484, 3.99364208, 8.64185631,
       9.68136177, 8.77843342, 8.07349922, 9.80650943, 1.18234631,
       4.78462409, 2.10586968, 2.13110648, 4.45772306, 2.7694983 ,
       7.61192806, 0.8625916 , 1.48446479, 9.97881665, 2.55960806,
       8.20268461, 0.47684678, 2.95886526, 5.66401659, 6.15267513,
       1.24069266, 6.38957581, 2.57710203, 3.54993113, 7.32367

In [22]:
input_buffer = allocate(128, np.float32)
output_buffer = allocate((8,8), np.float32)

In [23]:
np.copyto(input_buffer, data_input)

In [24]:
start_time = time.time()
data_send.transfer(input_buffer)
data_recv.transfer(output_buffer)
data_send.wait()
data_recv.wait()
end_time = time.time()
print(end_time - start_time)

0.003907918930053711


In [25]:
output_buffer

PynqBuffer([[245.47191 , 285.6463  , 200.76096 , 162.15904 , 250.32434 ,
             243.24034 , 241.82788 , 222.97485 ],
            [198.86652 , 247.13008 , 232.802   , 187.6673  , 208.3476  ,
             217.05783 , 251.55292 , 296.92288 ],
            [279.62238 , 347.85458 , 204.12567 , 210.64108 , 278.57907 ,
             283.82224 , 249.4034  , 262.12332 ],
            [178.19588 , 223.76376 , 226.69865 , 171.56378 , 186.39288 ,
             197.32797 , 208.25981 , 238.12361 ],
            [258.59918 , 345.55438 , 198.6958  , 201.18636 , 290.00223 ,
             276.6706  , 250.86046 , 250.88043 ],
            [152.60335 , 167.57912 , 107.01658 , 119.27225 , 124.48425 ,
             131.80704 , 129.52473 , 128.50114 ],
            [249.4471  , 304.99713 , 309.99228 , 253.37416 , 236.24306 ,
             259.0162  , 300.50748 , 358.07773 ],
            [143.7342  , 209.1575  , 120.67224 , 133.43256 , 166.07745 ,
             159.31152 , 134.50952 , 123.775566]], dtype=float32)

In [26]:
diff = np.dot(data_A, data_B) - output_buffer

In [27]:
rms_error = np.sqrt(np.mean(diff**2))

In [28]:
diff, rms_error

(PynqBuffer([[-2.16386468e-07, -1.56901496e-06, -4.34318355e-06,
               4.40611123e-06,  9.73393395e-06, -1.13212792e-05,
               3.14981364e-06, -4.23198941e-07],
             [-1.04537069e-05,  3.49000030e-06, -2.58982030e-06,
              -6.58033539e-06, -2.07414950e-05, -8.71035718e-06,
              -1.28918025e-05,  8.78674916e-06],
             [ 2.11784649e-05, -1.75932075e-05, -6.27106104e-06,
               2.73522772e-06, -1.81499861e-05,  3.36455759e-06,
              -1.55028648e-05,  3.64088055e-06],
             [ 2.33915719e-06, -5.98176553e-06, -1.21460036e-05,
              -5.67749288e-06,  9.96028527e-06, -1.06434825e-05,
              -9.82433971e-06, -1.49567541e-08],
             [-2.16164062e-06,  4.12459683e-05, -7.89616342e-06,
              -1.44735266e-05,  5.14634905e-07,  4.19381837e-06,
              -1.37354948e-05,  1.51376459e-05],
             [-9.97913705e-06,  1.41492762e-07, -6.13869622e-06,
               5.30292101e-06, -2.981921

In [34]:
%%time
np.dot(data_A, data_B)

CPU times: user 191 µs, sys: 0 ns, total: 191 µs
Wall time: 188 µs


array([[245.47190835, 285.6462997 , 200.76095147, 162.15904676,
        250.32435055, 243.24032987, 241.82788401, 222.97485309],
       [198.86650566, 247.13008467, 232.80199936, 187.66729078,
        208.34757447, 217.0578221 , 251.55290459, 296.92289087],
       [279.62239667, 347.85456615, 204.12566512, 210.6410855 ,
        278.57905289, 283.82223847, 249.4033811 , 262.12332517],
       [178.19587941, 223.76375745, 226.69864203, 171.56377606,
        186.39289326, 197.32796177, 208.25980158, 238.12361144],
       [258.59917997, 345.55442357, 198.69579289, 201.18634112,
        290.0022283 , 276.67059746, 250.86044464, 250.88044727],
       [152.6033378 , 167.57911696, 107.01657254, 119.27225262,
        124.48424995, 131.80703158, 129.52472418, 128.50114408],
       [249.44709354, 304.99713964, 309.99225724, 253.37416494,
        236.24305269, 259.0162231 , 300.50746301, 358.07773877],
       [143.73421125, 209.15749713, 120.67223874, 133.43253991,
        166.07745593, 159.3115216